<a href="https://colab.research.google.com/github/dpanagop/data_analytics_examples/blob/master/Markov_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text genaration using Markov chains

This is a short notebook that demonstrates how to genrate text using a transition matrix.

In [1]:
# Import libraries
import re
from random import random, sample

The transition matrix is defined in a class.
Actually, we are going to use a dictionary to create what is called a sparse matix. For example the table

|  | (am,a) | (am,happy) | (am,it) | (nice,thing)|
| --- | --- | --- | --- | --- |
| (i,am) | 1 | 2 | 0 | 0 |
((a,nice) | 0 | 0 | 0 | 3|


will be represented by the dictionary:

```{ "i,am": {"a":1, "happy":2} 
   , "a,nice": {"thing":3}
<br>}```

Note that 
- we are not adding entries for cells with zero value
- for in keys of the columns we are using only the next word 

PS. The careful reader will detect that we are not using propabilities in the matrix but instead a count. We will use this counts to calculate the corresponding propabilities.

In [2]:
class TransitionMatrix:
  """ This is the transition matrix class.

  Attributes: 
      SparseMatrix a dictionary in the form 
                 key: word_1, word_2
                 value: dictionary in form {word:integer, word:integer}
  Methods:
      next_word: add a tripple of word_1, word_2, word_3 in SparseMatrix
      next_word: given a tuple word_1,word_2 generate (next) word with probapilities according to 
      dictionary corresponding to key word_1,word_2 in SparseMatrix
  """
  def __init__(self):
    self.SparseMatrix={}
  def add_tripple(self, word1,word2,word3):
    """ for a given tripple word1,word2,word3
        check if word1,word2 is a key of the dictrionary
          if true and word_3 is a key in the corresponding entry increment value of key word_3 by one
          if true and word_3 is not a key then add word_3 as a key with value one
        if  word1,word2 is not a key, then add a key word_1,word_2 with value {word_3:1}
    """ 
    key=word1+","+word2
    if key in self.SparseMatrix:
      if word3 in self.SparseMatrix[key]:
        self.SparseMatrix[key][word3]=self.SparseMatrix[key][word3]+1
      else:
        self.SparseMatrix[key][word3]=1
    else:
      self.SparseMatrix[key]={word3:1}
  def next_word(self,word1,word2):
    """ generate next word for tuple word_1,word_2
        if word_1,word_2 is a key, then
           retrive the corresponding dictionary and based on it pick randomly a word
           see https://stackoverflow.com/questions/2570690/python-algorithm-to-randomly-select-a-key-based-on-proportionality-weight
        if word_1,word_2 is not a key, then select randomly from list [".","hence","thus","i"]
    """
    key=word1+","+word2
    if key in self.SparseMatrix:
      count=sum(self.SparseMatrix[key].values())
      rand_val = count*random()
      total = 0
      for word,idx in self.SparseMatrix[key].items():
        total += idx
        if rand_val <= total:
            return word
    else:
      return sample([".",",","and"],1)[0]

Example

In [3]:
# Assign classt to transitiontest and print contents
transitiontest=TransitionMatrix()
print(transitiontest.SparseMatrix)
# Add the tripple (i,am,a) 
transitiontest.add_tripple("i","am","a")
print(transitiontest.SparseMatrix)
# Increment count of tripple (i,am,a)
transitiontest.add_tripple("i","am","a")
print(transitiontest.SparseMatrix)
# Add trpple (i,am,happy)
transitiontest.add_tripple("i","am","happy")
print(transitiontest.SparseMatrix)

{}
{'i,am': {'a': 1}}
{'i,am': {'a': 2}}
{'i,am': {'a': 2, 'happy': 1}}


In [4]:
# Genrate next word
print(f'Next word for "i,am" :{transitiontest.next_word("i","am")}')
print(f'Next word for "you,are": {transitiontest.next_word("you","are")}. Note "you,are" is not contained in keys')

Next word for "i,am" :happy
Next word for "you,are": and. Note "you,are" is not contained in keys


## Text preprocessing
We will need to preprocess the text input.
Specificaly, we:
- convert to lowwercase
- remove special charaacters
- remove extra spacing

to this end, we define ```preprocess_sentence``` function

In [5]:
def preprocess_sentence(sentence):
  sentence=sentence.lower()
  sentence=re.sub(r"[^\w\d.!?\s]+",'',sentence)
  sentence=re.sub('([.,!?])', r' \1 ', sentence)
  sentence = re.sub('\s{2,}', ' ', sentence)
  return sentence

For creating the matrix, we are going to use texts from [Gutenberg Project](www.gutenberg.org).

In [6]:
!wget http://www.gutenberg.org/cache/epub/28/pg28.txt #Aesop's Fables, by Aesop
!wget https://www.gutenberg.org/files/1727/1727-0.txt #The Odyssey, by Homer
!wget http://www.gutenberg.org/files/6130/6130-0.txt #The Iliad, by Homer

--2020-12-05 05:24:55--  http://www.gutenberg.org/cache/epub/28/pg28.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86418 (84K) [text/plain]
Saving to: ‘pg28.txt.1’

pg28.txt.1          100%[===================>]  84.39K   310KB/s    in 0.3s    

2020-12-05 05:24:55 (310 KB/s) - ‘pg28.txt.1’ saved [86418/86418]

--2020-12-05 05:24:55--  https://www.gutenberg.org/files/1727/1727-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 718181 (701K) [text/plain]
Saving to: ‘1727-0.txt.1’

1727-0.txt.1        100%[===================>] 701.35K   672KB/s    in 1.0s    

2020-12-05 05:24:57 (672 KB/s) - ‘1727-0.txt.1

In [7]:
#Assign class TransitionMatrix to markov_matrix
Greeks_matrix=TransitionMatrix()
Greeks=['pg28.txt','1727-0.txt','6130-0.txt']
for FileName in Greeks:
  print(f'Processing {FileName}')
  # Open file
  with open(FileName) as f:
    content = f.readlines()
    #remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    content = [x.strip() for x in content if x!=""]
  # Process file
  for text in content:
    doc=preprocess_sentence(text)
    doc=doc.split()
    l=len(doc)
    for i in range(2,l):
      Greeks_matrix.add_tripple(doc[i-2],doc[i-1],doc[i])

Processing pg28.txt
Processing 1727-0.txt
Processing 6130-0.txt


Finally, generate text

In [11]:
word1="it"
word2="is"
story=word1+" "+word2
for i in range(50):
  new_word=Greeks_matrix.next_word(word1,word2)
  story=story+" "+new_word
  if new_word==".":
    print(story)
    story=""
  word1=word2
  word2=new_word
print(story)

it is enchanted and you can easily find another seat near telemachus he said to her ships and shelters there .
 the writer as stretching all and you do or cause to fight for the soil .
 he fenced the raft .
 at last they shall have plenty of it but by


In [12]:
!wget http://www.gutenberg.org/files/2600/2600-0.txt #War and Peace, by Leo Tolstoy
!wget http://www.gutenberg.org/files/1399/1399-0.txt #Anna Karenina, by Leo Tolstoy
!wget http://www.gutenberg.org/cache/epub/4761/pg4761.txt #The Cossacks, by Leo Tolstoy

--2020-12-05 05:25:20--  http://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359584 (3.2M) [text/plain]
Saving to: ‘2600-0.txt.2’

2600-0.txt.2        100%[===================>]   3.20M  1.99MB/s    in 1.6s    

2020-12-05 05:25:22 (1.99 MB/s) - ‘2600-0.txt.2’ saved [3359584/3359584]

--2020-12-05 05:25:22--  http://www.gutenberg.org/files/1399/1399-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2068079 (2.0M) [text/plain]
Saving to: ‘1399-0.txt.2’

1399-0.txt.2        100%[===================>]   1.97M  1.30MB/s    in 1.5s    

2020-12-05 05:25:24 (1.30 MB/s) - ‘1

In [13]:
#Assign class TransitionMatrix to markov_matrix
Tolstoy_matrix=TransitionMatrix()
Tolstoy=['2600-0.txt','1399-0.txt','pg4761.txt']
for FileName in Tolstoy:
  print(f'Processing {FileName}')
  # Open file
  with open(FileName) as f:
    content = f.readlines()
    #remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    content = [x.strip() for x in content if x!=""]
  # Process file
  for text in content:
    doc=preprocess_sentence(text)
    doc=doc.split()
    l=len(doc)
    for i in range(2,l):
      Tolstoy_matrix.add_tripple(doc[i-2],doc[i-1],doc[i])

Processing 2600-0.txt
Processing 1399-0.txt
Processing pg4761.txt


In [16]:
word1="it"
word2="is"
story=word1+" "+word2
for i in range(50):
  new_word=Tolstoy_matrix.next_word(word1,word2)
  story=story+" "+new_word
  if new_word==".":
    print(story)
    story=""
  word1=word2
  word2=new_word
print(story)

it is really ended ? i am an exception .
 .
 but being with nature seeing her patient smiling face and rigid .
 only when they came to a longstanding impression related to the war of 1815 alexander possesses all .
 he could not make out something black .
 pierre received one
